In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import re

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import unicodedata # Emoji ve özel semboller için
import nltk

nltk.download("stopwords")
stopwords = nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = text.lower()  # küçük harfe çevir
    text = re.sub(r'[^\w\s]', '', text)  # noktalama işaretlerini kaldır
    text = re.sub(r'\s+', ' ', text).strip()  # fazla boşlukları sadeleştir
    text = re.sub(r'<.*?>', '', text) # HTML etiketlerini kaldır
    text = re.sub(r"https?://\S+|www\S+", "", text) # URL'leri kaldır
    text = re.sub(r'#\S+', '', text) # Hashtag'leri kaldır
    text = re.sub(r"@\S+", "", text) # metindeki mentionleri kaldır
    text = " ".join([word for word in text.split() if word not in stopwords])
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/melikenurcaydan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # veya "facebook/bart-base"

max_input_length = 512
max_target_length = 128

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
def preprocess(batch):
    # article ve highlights birer liste olduğundan her eleman için ayrı ayrı temizle
    cleaned_articles = [clean_text(a) for a in batch["article"]]
    cleaned_summaries = [clean_text(s) for s in batch["highlights"]]

    # her biri için token işlemi uygula
    inputs = tokenizer(cleaned_articles, max_length=max_input_length, padding="max_length", truncation=True)
    targets = tokenizer(cleaned_summaries, max_length=max_target_length, padding="max_length", truncation=True)

    inputs["labels"] = targets["input_ids"]
    return inputs

In [6]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map: 100%|██████████| 11490/11490 [00:12<00:00, 939.22 examples/s]


In [7]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EarlyStoppingCallback

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small") #özetleme görevinde kullanılacak Transformer modeli yüklenir ve ardından Hugging Face'in Trainer API’si için eğitim konfigürasyonlarını tanımlanır. 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",                      # model ve log çıktıları buraya kaydedilecek
    evaluation_strategy="epoch",                 # her epoch sonunda değerlendirme
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,                          # hızlı prototipleme için 3 epoch
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",   # hangi metrik izlenecek
    greater_is_better=False,  
    logging_dir="./logs",
    logging_steps=100
)

In [17]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device) 

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(2000)),
    eval_dataset=tokenized_dataset["validation"].select(range(500)),
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # 2 epoch boyunca gelişme yoksa dur
)

In [18]:
trainer.train()

  4%|▍         | 100/2500 [00:56<20:52,  1.92it/s]

{'loss': 5.5415, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.2}


  8%|▊         | 200/2500 [01:53<37:25,  1.02it/s]

{'loss': 1.5303, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.4}


 12%|█▏        | 300/2500 [03:41<32:27,  1.13it/s]

...

                                                   
100%|██████████| 2500/2500 [43:41<00:00,  1.26it/s]

{'eval_loss': 0.8460965156555176, 'eval_runtime': 30.579, 'eval_samples_per_second': 16.351, 'eval_steps_per_second': 4.088, 'epoch': 5.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 2500/2500 [43:42<00:00,  1.05s/it]

{'train_runtime': 2622.6072, 'train_samples_per_second': 3.813, 'train_steps_per_second': 0.953, 'train_loss': 1.3373460021972656, 'epoch': 5.0}


TrainOutput(global_step=2500, training_loss=1.3373460021972656, metrics={'train_runtime': 2622.6072, 'train_samples_per_second': 3.813, 'train_steps_per_second': 0.953, 'train_loss': 1.3373460021972656, 'epoch': 5.0})

In [25]:
import evaluate
rouge = evaluate.load("rouge")

In [26]:
model.to(device)
model.eval() 

for i in range(5):
    article = tokenized_dataset["test"][i]["article"]
    reference = tokenized_dataset["test"][i]["highlights"]

    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding="max_length", max_length=512) #Haber metni, tokenizer ile modele uygun formata dönüştürülüyor (tensor olarak).
    input_ids = inputs["input_ids"].to(device)  # giriş verisini MPS'e taşı

    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"\nHaber {i+1}")
    print("Model Özeti:\n", generated_summary)
    print("Gerçek Özet:\n", reference)



Haber 1
Model Özeti:
 the 123rd member of the international Criminal Court is a step that gives the court jurisdiction over alleged crimes in palestinians. the ICC opened a preliminary examination into the situation in Palestinian territories.
Gerçek Özet:
 Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Haber 2
Model Özeti:
 Theia is a friendly white-and-black bully breed mix now named theia. she was found four days after being hit by a car and buried in a field. the dog has been receiving care at the Veterinary Teaching Hospital. she suffered dislocated jaw, leg injuries and a caved-in sinus cavity.
Gerçek Özet:
 Theia, a bully breed mix, was apparently hit by a car, whacked with a hammer and buried in a field .
"She's a true miracle dog and she deserves a good life," says Sara Mellado, who is looki

In [27]:
generated_summaries = []
reference_summaries = []

for i in range(5):
    article = tokenized_dataset["test"][i]["article"]
    reference = tokenized_dataset["test"][i]["highlights"]

    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    input_ids = inputs["input_ids"].to(device)
    model.to(device)
    model.eval()

    summary_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    generated = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    generated_summaries.append(generated)
    reference_summaries.append(reference)

# ROUGE hesapla
rouge_scores = rouge.compute(predictions=generated_summaries, references=reference_summaries)

# Sonuçları yazdır
print("ROUGE-1:", rouge_scores["rouge1"])
print("ROUGE-2:", rouge_scores["rouge2"])
print("ROUGE-L:", rouge_scores["rougeL"])


ROUGE-1: 0.3342826888502629
ROUGE-2: 0.13014166959372436
ROUGE-L: 0.2439524660081373
